In [ ]:
!pip install transformers

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import torch

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device. {device}')

Using device. cuda


In [ ]:
model_name = "ytu-ce-cosmos/turkish-gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 1280)

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
dataset_openorca = load_dataset("ucekmez/OpenOrca-tr")

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
        def __init__(self, tokenizer, data, block_size):
            self.tokenizer = tokenizer
            self.data = data['train']
            self.block_size = block_size

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            item = self.data[idx]
            tokenized_inputs = self.tokenizer(
                item["question"],
                item["response"],
                truncation=True,
                padding="max_length",
                max_length=self.block_size,
                return_tensors="pt"
            )
            tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()
            return tokenized_inputs



In [ ]:
tokenized_dataset = CustomDataset(tokenizer, dataset_openorca, 128)

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import TrainingArguments, Trainer
import accelerate
training_args = TrainingArguments(
    output_dir="finetuned-ytu",
    evaluation_strategy = "epoch" or "steps",
    learning_rate=4e-5,
    per_device_train_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit = 2,
    load_best_model_at_end=False
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=None,
    data_collator=data_collator,
)

In [ ]:
trainer.train(resume_from_checkpoint = True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Epoch,Training Loss,Validation Loss


Görüntülenen çıkış son 5000 satıra kısaltıldı.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.download('/content/finetuned-ytu')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/nurke/finetuned-ytu/commit/842ae0df87a40666583f2431658fed20eb2222a3', commit_message='End of training', commit_description='', oid='842ae0df87a40666583f2431658fed20eb2222a3', pr_url=None, pr_revision=None, pr_num=None)